### Questions
* getting a handle on api workflow

### Objectives
YWBAT 
* hide api permissions in your notebook
* access api and use it in data collection

### Outline
* build a twitter data collection tool
* store our tweets in a mongodb

In [1]:
import json
import pandas as pd
import numpy as np
import tweepy as tw

from pymongo import MongoClient

import matplotlib.pyplot as plt

#### Loading in my twitter info

In [2]:
with open("/Users/rafael/.ssh/twitter_app00.json") as f:
    d = json.load(f)

In [20]:
oauth = tw.OAuthHandler(consumer_key=d["consumer_key"], consumer_secret=d["consumer_secret"])
oauth.set_access_token(key=d["access_token"], secret=d["access_token_secret"])

In [21]:
api = tw.API(auth_handler=oauth)

## What can I do with Twitter's API?
* udpate a status

In [39]:
def update_statuses(api, status_list):
    for status in status_list:
        api.update_status(status)
    pass


def delete_recent_tweet(api):
    last_tweet_id = api.me()._json["status"]["id"]
    print("found id")
    print("deleting that ish")
    api.destroy_status(last_tweet_id)
    print("deleted...whew....")
    pass

In [50]:
# search stuff
results = api.search("#rickandmortyseason4", tweet_mode='extended')

In [52]:
len(results)

14

boooooo!!!! only 14 results? Dude, I want thousands....

In [93]:
class MyStreamListener(tw.StreamListener):
    
    
    def on_status(self, status):
#         try:
#             print(status.extended_tweet["full_text"])
#         except:
#             print('---------not extended status---------')
#             print(status.text)
        print("inserted tweet into collection")
        got_collection.insert_one(status._json)
        pass
        
     
    def on_error(self, status_code):
        if status_code == 420:
            #returning False in on_error disconnects the stream
            return False

        # returning non-False reconnects the stream, with backoff.

In [94]:
# Make a streamer object
listener = MyStreamListener()

# make a stream using our listener
stream = tw.Stream(auth=oauth, listener=listener)

# Connect to Mongo and Create 'Table' (collection) for got tweets

In [95]:
client = MongoClient(host='localhost', port=27017)
tweets = client["tweets"]

In [96]:
got_collection = tweets["got_collection"]

In [97]:
rick_and_morty_list = ["@RickAndMorty", "#rickandmortyseason4", "#rickandmorty", '#wubbalubbadubdub', '#picklerick', '#tinyrick', 'rick and morty']
got = ["game of thrones", "#got"]
stream.filter(track=got, is_async=True)

inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted

inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted

inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted

inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted

inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted tweet into collection
inserted

Exception in thread Thread-7:
Traceback (most recent call last):
  File "/anaconda3/lib/python3.7/http/client.py", line 544, in _get_chunk_left
    chunk_left = self._read_next_chunk_size()
  File "/anaconda3/lib/python3.7/http/client.py", line 511, in _read_next_chunk_size
    return int(line, 16)
ValueError: invalid literal for int() with base 16: b''

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/anaconda3/lib/python3.7/http/client.py", line 576, in _readinto_chunked
    chunk_left = self._get_chunk_left()
  File "/anaconda3/lib/python3.7/http/client.py", line 546, in _get_chunk_left
    raise IncompleteRead(b'')
http.client.IncompleteRead: IncompleteRead(0 bytes read)

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/anaconda3/lib/python3.7/site-packages/urllib3/response.py", line 360, in _error_catcher
    yield
  File "/anaconda3/lib/python3.7/site-pa

### Assessment